<a href="https://colab.research.google.com/github/samanth31/XSentimental-analysis/blob/main/X_Sentiment_Analysis_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json !/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kazanova/sentiment140

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 90% 73.0M/80.9M [00:01<00:00, 30.6MB/s]
100% 80.9M/80.9M [00:01<00:00, 43.6MB/s]


In [ ]:
from zipfile import ZipFile
data='/content/sentiment140.zip';
with ZipFile(data,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv",encoding='ISO  8859 1');

In [ ]:
data.shape

(1599999, 6)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [ ]:
coloumn_names=['target','id','date','flag','user','text']
data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv",names=coloumn_names,encoding='latin1')

In [ ]:
data.shape

(1600000, 6)

In [ ]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
data.dropna(inplace=True)
data.isnull().sum()


,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
# data['target'].value_counts()
target_value_counts = data['target'].value_counts()
if 4 in target_value_counts.index:
    print("Value 4 exists in the 'target' column.")
else:
    print("Value 4 does not exist in the 'target' column.")


Value 4 exists in the 'target' column.


In [ ]:
data.replace(4,1,inplace=True)

data['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

ValueError: mount failed

In [ ]:
# Check if the processed file exists
processed_file = '/content/drive/MyDrive/processed_tweets.csv'

if os.path.exists(processed_file):
    data = pd.read_csv(processed_file)
    print("Loaded processed data from Google Drive.")
else:
    # Perform stemming directly on 'text' column
    port_stem = PorterStemmer()
    data['text'] = data['text'].apply(lambda x: ' '.join([port_stem.stem(word) for word in x.split()]))

    # Further processing with the stemming function
    def stemming(content):
        sc = re.sub('[^A-Za-z]', ' ', content)
        sc = sc.lower()
        sc = sc.split()
        sc = [port_stem.stem(word) for word in sc if not word in stopwords.words('english')]
        sc = ' '.join(sc)
        return sc
 # Process data in batches
batch_size = 10000  # Adjust batch size as needed
num_batches = len(data) // batch_size + 1

for i in range(num_batches):
    batch_data = data[i * batch_size:(i + 1) * batch_size]
    batch_data['sc'] = batch_data['text'].apply(stemming)

    # Save each batch to a CSV file
    batch_file = f'/content/drive/MyDrive/processed_tweets_batch_{i}.csv'
    batch_data.to_csv(batch_file, index=False)
    print(f"Processed and saved batch {i + 1}/{num_batches}")

    # data['sc'] = data['text'].apply(stemming)

    # # Save the processed data to Google Drive
    # data.to_csv(processed_file, index=False)
    # print("Processed data saved to Google Drive.")


<ipython-input-21-47491e4c02c4>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_data['sc'] = batch_data['text'].apply(stemming)


Processed and saved batch 1/161


<ipython-input-21-47491e4c02c4>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_data['sc'] = batch_data['text'].apply(stemming)


Processed and saved batch 2/161


<ipython-input-21-47491e4c02c4>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_data['sc'] = batch_data['text'].apply(stemming)


...

<ipython-input-21-47491e4c02c4>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_data['sc'] = batch_data['text'].apply(stemming)


Processed and saved batch 158/161


<ipython-input-21-47491e4c02c4>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_data['sc'] = batch_data['text'].apply(stemming)


Processed and saved batch 159/161
Processed and saved batch 160/161
Processed and saved batch 161/161


<ipython-input-21-47491e4c02c4>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_data['sc'] = batch_data['text'].apply(stemming)


In [ ]:
# Directory where the batch files are saved
batch_files_dir = '/content/drive/MyDrive/'

# List all batch files (adjust the pattern if necessary)
batch_files = [f for f in os.listdir(batch_files_dir) if f.startswith('processed_tweets_batch_') and f.endswith('.csv')]

# Sort files to ensure they are processed in order (if needed)
batch_files.sort()

# List to hold DataFrames
dfs = []

# Read each batch file into a DataFrame and append to the list
for file in batch_files:
    file_path = os.path.join(batch_files_dir, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames
combined_data = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a single CSV file
combined_data.to_csv('/content/drive/MyDrive/combined_processed_tweets.csv', index=False)

print("All batch files have been merged and saved as 'combined_processed_tweets.csv'.")


NameError: name 'os' is not defined

In [ ]:
port_stem = PorterStemmer()
data['text'] = data['text'].apply(lambda x: ' '.join([port_stem.stem(word) for word in x.split()]))
data['text'].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he can't updat hi facebook by te...
2    @kenichan i dive mani time for the ball. manag...
3         my whole bodi feel itchi and like it on fire
4    @nationwideclass no, it' not behav at all. i'm...
Name: text, dtype: object

In [ ]:
def stemming(content):
  sc=re.sub('[^A-Za-z]',' ',content);
  sc=sc.lower();
  sc=sc.split();
  sc=[port_stem.stem(word) for word in sc if not word in stopwords.words('english')]
  sc=' '.join(sc)
  return sc;

In [ ]:
data['sc'] = data['text'].apply(stemming)

In [ ]:
data.head()

In [ ]:
print(data['text'])

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          is upset that he can't updat hi facebook by te...
2          @kenichan i dive mani time for the ball. manag...
3               my whole bodi feel itchi and like it on fire
4          @nationwideclass no, it' not behav at all. i'm...
                                 ...                        
1599995    just woke up. have no school is the best feel ...
1599996    thewdb.com - veri cool to hear old walt interv...
1599997    are you readi for your mojo makeover? ask me f...
1599998    happi 38th birthday to my boo of alll time!!! ...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: text, Length: 1600000, dtype: object


In [ ]:
print(data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X = data['text'].values
Y = data['target'].values

In [ ]:
print(X)

["@switchfoot http://twitpic.com/2y1zl - awww, that' a bummer. you shoulda got david carr of third day to do it. ;d"
 "is upset that he can't updat hi facebook by text it... and might cri as a result school today also. blah!"
 '@kenichan i dive mani time for the ball. manag to save 50% the rest go out of bound'
 ... 'are you readi for your mojo makeover? ask me for detail'
 'happi 38th birthday to my boo of alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=3)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['the arm just fell of my comput chair now what am i go to lean on'
 '@chrishillfan you got media next, lucky!'
 'it suck havin to watch yur mother be sick and cnt help her' ...
 '@melosm oooop i did it again sorri for the non exist miss call...'
 "@frowzl accord to jï¿½rg kachelmann it'll start rain tomorrow, 5 o'clock in the morn ;) let' hope he' right! ja komm auch mit"
 "@alcarlton lol, no idea mate, i'd go buy more clothes, much simpler option!"]


In [ ]:
print(X_test)

["i hate when the dog bark at someth when noth happened, it' scari time to tri to sleep."
 '@willie_day26 me either but i will support it cuz i like you guy'
 '@jordalynn yah it wa pretti mess up' ...
 'use a spare usb flash drive as virtual ram for my pc. now it got 5.6 gb of memori'
 '@sistatreen so jealou i miss the beach down there'
 '@sakura_kurosaki no prob, i hope you have a good one &lt;3 ooh, and what kind of cake?! i love cake.']


In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 388714)	0.15523097117665094
  (0, 305327)	0.461691528661243
  (0, 516748)	0.10560637214385764
  (0, 212970)	0.16544687785669043
  (0, 53187)	0.2114378830110619
  (0, 549875)	0.20088907837000758
  (0, 381611)	0.18086337231538163
  (0, 115294)	0.4099591661851456
  (0, 130707)	0.32355788094853183
  (0, 364734)	0.128773097507387
  (0, 385333)	0.1519944185459127
  (0, 191066)	0.34575915038844574
  (0, 277418)	0.16706888683088467
  (0, 64808)	0.36695217143528847
  (0, 505512)	0.10903490163721127
  (1, 320810)	0.41770281008005516
  (1, 374025)	0.2726908268058994
  (1, 338572)	0.4143842106151637
  (1, 215240)	0.21381725298293178
  (1, 568747)	0.14358096529632905
  (1, 121812)	0.7163214991548106
  (2, 229546)	0.22932165723399553
  (2, 229124)	0.2593761528319698
  (2, 127813)	0.40763623306071234
  (2, 56622)	0.12583128005561484
  :	:
  (1279998, 423883)	0.1382968431221647
  (1279998, 307776)	0.1327107019253116
  (1279998, 226574)	0.11813912563262581
  (1279998, 314812)	0.11521676884433162


In [ ]:
print(X_test)

  (0, 550397)	0.39017832564188143
  (0, 522645)	0.21097693233261539
  (0, 516748)	0.18604314497227198
  (0, 514317)	0.17632100488889962
  (0, 505512)	0.09604153423950827
  (0, 475419)	0.24406401102816608
  (0, 470594)	0.20567420153990926
  (0, 451755)	0.3291852031657196
  (0, 381085)	0.2532415310629075
  (0, 251690)	0.11102497834918913
  (0, 225344)	0.2183238468552112
  (0, 224008)	0.34380462107613374
  (0, 161872)	0.27269016616164177
  (0, 76999)	0.42180184073499927
  (0, 68672)	0.15326776799398614
  (1, 568747)	0.15182936534987881
  (1, 552945)	0.5523155357795776
  (1, 552713)	0.23523991714884177
  (1, 492257)	0.3833912475705859
  (1, 338092)	0.17565453060603356
  (1, 310124)	0.21082454347280857
  (1, 251690)	0.141564680677873
  (1, 220330)	0.29697577103959094
  (1, 173846)	0.35390673854879834
  (1, 140221)	0.3557735104125364
  :	:
  (319997, 194634)	0.30275601305609706
  (319997, 166337)	0.22568241796567862
  (319997, 66102)	0.1720201145196952
  (319998, 509353)	0.33392459094818194


In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print("Accuracy score of training data : ",training_data_accuracy)

Accuracy score of training data :  0.8302453125


In [ ]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [ ]:
print("Accuracy score of testing data : ",testing_data_accuracy)

Accuracy score of testing data :  0.80358125


In [ ]:
import pickle

In [ ]:
filename = 'sentiment model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model = pickle.load(open('/content/sentiment model.sav','rb'))

In [ ]:
X_new = ["i will love you"]
X_new_transformed=vectorizer.transform(X_new)
prediction = loaded_model.predict(X_new_transformed)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

[1]
Positive Tweet
